#code aimed at generating a content based recommmender system
#using the Xwines dataset

In [8]:
##standard imports
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix as csr
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from matplotlib import pyplot as plt
import cv2 as cv

In [49]:
##importing the data
data   = pd.read_csv("XWines_Test_100_wines.csv", low_memory=False, encoding="utf-8", memory_map=True)
print(type(data))
print(data)
#define scalar and encoder
encoder = OneHotEncoder()
scaler = MinMaxScaler()

<class 'pandas.core.frame.DataFrame'>
    WineID                                  WineName          Type  \
0   100062                             Origem Merlot           Red   
1   100191                        Reserva Chardonnay         White   
2   101847          Dona Antonia Porto Reserva Tawny  Dessert/Port   
3   102055                            Fine Ruby Port  Dessert/Port   
4   102079                                 Maré Alta         White   
..     ...                                       ...           ...   
95  195476  Cuvée Alexander Rose de Pinot Extra Brut     Sparkling   
96  195831                            Kormilitsa Red           Red   
97  196718                          Select Rosé Brut     Sparkling   
98  196838                                  Kallisto         White   
99  198580                     The Falcon Pinot Gris         White   

           Elaborate                                             Grapes  \
0      Varietal/100%                          

In [ ]:
##data cleaning

In [38]:
#numerical data
#WineID,WineName,Type,Elaborate,Grapes,Harmonize,ABV,Body,Acidity,Code,Country,RegionID,RegionName,WineryID,WineryName,Website,Vintages

numerical_data=['ABV']
encoded_data=["Type","Acidity","Grapes","Country"]
all_scaled=[len(numerical_data)]

for i in range(len(numerical_data)):
    all_scaled[i]=scaler.fit_transform(data[[numerical_data[i]]])

encoded_cols = encoder.fit_transform(data[encoded_data]).toarray()

##change in future
feature_matrix = np.hstack((encoded_cols,all_scaled[0]))

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate cosine similarity between wines
cosine_sim = cosine_similarity(encoded_cols)
print(feature_matrix)
print(encoded_cols)

[[0.         0.         1.         ... 0.         0.         0.20454545]
 [0.         0.         0.         ... 0.         0.         0.20454545]
 [0.         1.         0.         ... 0.         0.         0.36363636]
 ...
 [0.         0.         0.         ... 0.         0.         0.19318182]
 [0.         0.         0.         ... 0.         0.         0.20454545]
 [0.         0.         0.         ... 0.         0.         0.20454545]]
[[0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [51]:
def get_recommendations(wine_name, data, cosine_sim):
    # Get the index of the wine that matches the wine_name
   
    idx = data.index[data['WineName'] == wine_name].tolist()[0]
    print("yep")
    # Get the similarity scores for this wine
    sim_scores = list(enumerate(cosine_sim[idx]))
    
    # Sort wines by similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # Get the indices of the top 5 most similar wines
    sim_scores = sim_scores[1:6]
    wine_indices = [i[0] for i in sim_scores]
    
    # Return the top 5 most similar wines
    return data['WineName'].iloc[wine_indices]

# Example usage:
print(get_recommendations('Origem Merlot', data, cosine_sim))

yep
18               Merlot Reserve Trocken
1                    Reserva Chardonnay
6     Pátria Alentejo Selection Reserva
8                 Calda Bordaleza Tinto
9                        Regional Tinto
Name: WineName, dtype: object
